In [1]:
import random
import json
import pickle
import numpy as np
import tensorflow as tf
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer
from tensorflow.keras.optimizers import SGD

In [2]:
lemmatizer = WordNetLemmatizer()
# Initialize the French stemmer
fr_stemmer = SnowballStemmer('french')
# Load and parse the intents JSON file in French
with open('mbi_questions_frensh.json', 'r', encoding="utf8") as file:
    intents = json.load(file)

In [3]:
words = []
classes = []
documents = []
ignoreLetters = ['?', '!', '.', ',']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        # Convert to lowercase before tokenizing
        wordList = nltk.word_tokenize(pattern.lower())
        words.extend(wordList)
        documents.append((wordList, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# Convert words to lowercase and lemmatize, ignoring specified characters
words = [fr_stemmer.stem(w.lower()) for w in words if w not in ignoreLetters]
words = sorted(set(words))

classes = sorted(set(classes))

# Save words and classes for future use
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

# Prepare training data
training = []
outputEmpty = [0] * len(classes)

In [4]:
for document in documents:
    bag = []
    wordPatterns = document[0]
    # Convert each word to lowercase and lemmatize
    wordPatterns = [lemmatizer.lemmatize(word.lower()) for word in wordPatterns]
    for word in words:
        bag.append(1) if word in wordPatterns else bag.append(0)

    outputRow = list(outputEmpty)
    outputRow[classes.index(document[1])] = 1
    training.append(bag + outputRow)

random.shuffle(training)
training = np.array(training)

# Split data into training inputs and outputs
trainX = training[:, :len(words)]
trainY = training[:, len(words):]

# Build and compile the model
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(128, input_shape=(len(trainX[0]),), activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(len(trainY[0]), activation='softmax'))

# Define optimizer and compile model
optimizer = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
hist = model.fit(np.array(trainX), np.array(trainY), epochs=200, batch_size=5, verbose=1)

Epoch 1/200
11/11 [==============================] - 0s 2ms/step - loss: 2.8673 - accuracy: 0.0392
Epoch 2/200
11/11 [==============================] - 0s 2ms/step - loss: 2.7210 - accuracy: 0.1373
Epoch 3/200
11/11 [==============================] - 0s 1ms/step - loss: 2.6066 - accuracy: 0.2157
Epoch 4/200
11/11 [==============================] - 0s 1ms/step - loss: 2.4941 - accuracy: 0.2353
Epoch 5/200
11/11 [==============================] - 0s 1ms/step - loss: 2.4330 - accuracy: 0.3529
Epoch 6/200
11/11 [==============================] - 0s 1ms/step - loss: 2.2696 - accuracy: 0.3137
Epoch 7/200
11/11 [==============================] - 0s 1ms/step - loss: 2.1839 - accuracy: 0.4314
Epoch 8/200
11/11 [==============================] - 0s 992us/step - loss: 2.1017 - accuracy: 0.3922
Epoch 9/200
11/11 [==============================] - 0s 1ms/step - loss: 1.9337 - accuracy: 0.4118
Epoch 10/200
11/11 [==============================] - 0s 1ms/step - loss: 1.9314 - accuracy: 0.4902
Epoch 1

11/11 [==============================] - 0s 1ms/step - loss: 0.3808 - accuracy: 0.8824
Epoch 83/200
11/11 [==============================] - 0s 1ms/step - loss: 0.2619 - accuracy: 0.9020
Epoch 84/200
11/11 [==============================] - 0s 1ms/step - loss: 0.2570 - accuracy: 0.8824
Epoch 85/200
11/11 [==============================] - 0s 1ms/step - loss: 0.2017 - accuracy: 0.9412
Epoch 86/200
11/11 [==============================] - 0s 1ms/step - loss: 0.2230 - accuracy: 0.9216
Epoch 87/200
11/11 [==============================] - 0s 1ms/step - loss: 0.2856 - accuracy: 0.9020
Epoch 88/200
11/11 [==============================] - 0s 1ms/step - loss: 0.1869 - accuracy: 0.9216
Epoch 89/200
11/11 [==============================] - 0s 1000us/step - loss: 0.2452 - accuracy: 0.9020
Epoch 90/200
11/11 [==============================] - 0s 1ms/step - loss: 0.3171 - accuracy: 0.8627
Epoch 91/200
11/11 [==============================] - 0s 1ms/step - loss: 0.2873 - accuracy: 0.8824
Epoch 92/2

11/11 [==============================] - 0s 1ms/step - loss: 0.2951 - accuracy: 0.8824
Epoch 164/200
11/11 [==============================] - 0s 1ms/step - loss: 0.1404 - accuracy: 0.9412
Epoch 165/200
11/11 [==============================] - 0s 1ms/step - loss: 0.2299 - accuracy: 0.9412
Epoch 166/200
11/11 [==============================] - 0s 1ms/step - loss: 0.2107 - accuracy: 0.9020
Epoch 167/200
11/11 [==============================] - 0s 1ms/step - loss: 0.1684 - accuracy: 0.9216
Epoch 168/200
11/11 [==============================] - 0s 1ms/step - loss: 0.2023 - accuracy: 0.9216
Epoch 169/200
11/11 [==============================] - 0s 1ms/step - loss: 0.2260 - accuracy: 0.9020
Epoch 170/200
11/11 [==============================] - 0s 1ms/step - loss: 0.2974 - accuracy: 0.9020
Epoch 171/200
11/11 [==============================] - 0s 1ms/step - loss: 0.1688 - accuracy: 0.9216
Epoch 172/200
11/11 [==============================] - 0s 1ms/step - loss: 0.2285 - accuracy: 0.9020
Epoc

In [5]:
# Save the trained model
model.save('chatbot_model_fr.keras', hist)
print('Done')

Done
